# Assignment 3 – Using the A* Algorithm
## Task 1.1
An implementation of the A* algorithm has been lifted from [here](https://github.com/jrialland/python-astar). The file of interest is `astar/__init__.py` where the `AStar` class is implemented.

In [ ]:
# %load astar/__init__.py
""" generic A-Star path searching algorithm """

from abc import ABCMeta, abstractmethod
from heapq import heappush, heappop

__author__ = "Julien Rialland"
__copyright__ = "Copyright 2012-2017, J.Rialland"
__license__ = "BSD"
__version__ = "0.9"
__maintainer__ = __author__
__email__ = ''.join(map(chr, [106, 117, 108, 105, 101, 110, 46, 114, 105, 97, 108, 108, 97, 110, 100, 64, 103, 109, 97, 105, 108, 46, 99, 111, 109]))
__status__ = "Production"




Infinite = float('inf')

class AStar:
    __metaclass__ = ABCMeta
    __slots__ = ()

    class SearchNode:
        __slots__ = ('data', 'gscore', 'fscore',
                     'closed', 'came_from', 'out_openset')

        def __init__(self, data, gscore=Infinite, fscore=Infinite):
            self.data = data
            self.gscore = gscore
            self.fscore = fscore
            self.closed = False
            self.out_openset = True
            self.came_from = None

        def __lt__(self, b):
            return self.fscore < b.fscore

    class SearchNodeDict(dict):

        def __missing__(self, k):
            v = AStar.SearchNode(k)
            self.__setitem__(k, v)
            return v

    @abstractmethod
    def heuristic_cost_estimate(self, current, goal):
        """Computes the estimated (rough) distance between a node and the goal, this method must be implemented in a subclass. The second parameter is always the goal."""
        raise NotImplementedError

    @abstractmethod
    def distance_between(self, n1, n2):
        """Gives the real distance between two adjacent nodes n1 and n2 (i.e n2 belongs to the list of n1's neighbors).
           n2 is guaranteed to belong to the list returned by the call to neighbors(n1).
           This method must be implemented in a subclass."""
        raise NotImplementedError

    @abstractmethod
    def neighbors(self, node):
        """For a given node, returns (or yields) the list of its neighbors. this method must be implemented in a subclass"""
        raise NotImplementedError

    def is_goal_reached(self, current, goal):
        """ returns true when we can consider that 'current' is the goal"""
        return current == goal

    def reconstruct_path(self, last, reversePath=False):
        def _gen():
            current = last
            while current:
                yield current.data
                current = current.came_from
        if reversePath:
            return _gen()
        else:
            return reversed(list(_gen()))

    def astar(self, start, goal, reversePath=False):
        if self.is_goal_reached(start, goal):
            return [start]
        searchNodes = AStar.SearchNodeDict()
        startNode = searchNodes[start] = AStar.SearchNode(
            start, gscore=.0, fscore=self.heuristic_cost_estimate(start, goal))
        openSet = []
        heappush(openSet, startNode)
        while openSet:
            current = heappop(openSet)
            if self.is_goal_reached(current.data, goal):
                return self.reconstruct_path(current, reversePath)
            current.out_openset = True
            current.closed = True
            for neighbor in [searchNodes[n] for n in self.neighbors(current.data)]:
                if neighbor.closed:
                    continue
                tentative_gscore = current.gscore + \
                    self.distance_between(current.data, neighbor.data)
                if tentative_gscore >= neighbor.gscore:
                    continue
                neighbor.came_from = current
                neighbor.gscore = tentative_gscore
                neighbor.fscore = tentative_gscore + \
                    self.heuristic_cost_estimate(neighbor.data, goal)
                if neighbor.out_openset:
                    neighbor.out_openset = False
                    heappush(openSet, neighbor)
        return None

def find_path(start, goal, neighbors_fnct, reversePath=False, heuristic_cost_estimate_fnct=lambda a, b: Infinite, distance_between_fnct=lambda a, b: 1.0, is_goal_reached_fnct=lambda a, b: a == b):
    """A non-class version of the path finding algorithm"""
    class FindPath(AStar):
        def heuristic_cost_estimate(self, current, goal):
            return heuristic_cost_estimate_fnct(current, goal)
        def distance_between(self, n1, n2):
            return distance_between_fnct(n1, n2)
        def neighbors(self, node):
            return neighbors_fnct(node)
        def is_goal_reached(self, current, goal):
            return is_goal_reached_fnct(current, goal)
    return FindPath().astar(start, goal, reversePath)

__all__ = ['AStar', 'find_path']


The A* algorithm has been used in the mazesolver module, specifically in `mazesolver.MazeSolver`. The source code follows:

In [ ]:
# %load mazesolver/__init__.py
from typing import Tuple, List
import numpy as np

from astar import AStar


class MazeSolver(AStar):
    """
    Each node is represented by its index (x,y) in the maze array
    """
    REACHABLE_NODES = ['A', 'B', '.']

    def __init__(self, maze: str) -> None:
        lines = maze.split('\n')

        # All lines need to be of the same width as the first line
        self.width = len(lines[0])

        items = [
            [c for c in line]
            for line
            in lines
            if len(line) == self.width
        ]
        self.height = len(items)

        self.maze = np.array(items)

        assert self.height, self.width == self.maze.shape

        start_index = np.where(self.maze == 'A')
        self.start = (start_index[0][0], start_index[1][0],)

        goal_index = np.where(self.maze == 'B')
        self.goal = (goal_index[0][0], goal_index[1][0],)

    def neighbors(self, node: Tuple):
        x, y = node
        nnodes = ((x - 1, y), (x, y - 1), (x + 1, y), (x, y + 1))
        return tuple(nnode for nnode in nnodes if self.reachable(nnode))

    def reachable(self, node: Tuple) -> bool:
        x, y = node

        if not ((0 <= x < self.width) and (0 <= y < self.height)):
            return False
        else:
            return self.maze[x, y] in self.REACHABLE_NODES

    def distance_between(self, n1: Tuple, n2: Tuple) -> int:
        # Neighbouring nodes are always 1 units apart
        return 1

    def heuristic_cost_estimate(self, current: Tuple, goal: Tuple) -> int:
        return abs(goal[0] - current[0]) + abs(goal[1] - current[1])

    def solve(self) -> List[Tuple]:
        # self.astar() returns a generator, so it is cast into a list
        self.path = list(self.astar(self.start, self.goal))

        self.solved_maze = self.maze.copy()
        for position in self.path:
            self.solved_maze[position] = 'O'

        return self.path


all = ['MazeSolver']


The validity of the code has been confirmed by the tests placed in `mazesolver.tests.test_maze_solver.TestMazeSolver`.

In [ ]:
# %load mazesolver/tests/test_maze_solver.py
import numpy as np

from mazesolver import MazeSolver


class TestMazeSolver:
    def test_init_with_file(self):
        with open('mazesolver/tests/boards/board-1-1.txt') as f:
            ms = MazeSolver(f.read())

        assert ms.maze.shape == (7, 20)

    def test_simple_maze_string(self):
        maze = '.#\nAB'
        ms = MazeSolver(maze)

        assert np.array_equal(ms.maze, np.array([['.', '#'], ['A', 'B']]))

    def test_dimensions_of_maze(self):
        maze = '.#.\nAB.'
        ms = MazeSolver(maze)

        assert ms.width == 3
        assert ms.height == 2

    def test_start_and_goal_index(self):
        maze = '.#.\nAB.'
        ms = MazeSolver(maze)

        assert ms.start == (1, 0,)
        assert ms.goal == (1, 1,)

    def test_reachable(self):
        maze = 'A#B\n.#.\n...'
        ms = MazeSolver(maze)

        # Check all points within maze
        assert ms.reachable((0, 0)) is True
        assert ms.reachable((0, 1)) is False
        assert ms.reachable((0, 2)) is True

        assert ms.reachable((1, 0)) is True
        assert ms.reachable((1, 1)) is False
        assert ms.reachable((1, 2)) is True

        assert ms.reachable((2, 0)) is True
        assert ms.reachable((2, 1)) is True
        assert ms.reachable((2, 2)) is True

        # Out of bounds
        assert ms.reachable((-1, 0)) is False
        assert ms.reachable((0, -1)) is False
        assert ms.reachable((3, -1)) is False
        assert ms.reachable((0, 3)) is False

    def test_neighbours(self):
        maze = 'A#B\n.#.\n...'
        ms = MazeSolver(maze)

        assert ms.neighbors((0, 0)) == ((1, 0),)

    def test_distance_between_adjacent_nodes(self, maze_solver):
        assert maze_solver.distance_between(None, None) == 1

    def test_heuristic_cost_estimate(self, maze_solver):
        f = maze_solver.heuristic_cost_estimate

        assert f((0, 0), (0, 0)) == 0
        assert f((0, 1), (0, 0)) == 1
        assert f((0, 0), (0, 1)) == 1
        assert f((3, 3), (0, 1)) == 5

    def test_solve(self, maze_solver):
        solution = [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (1, 2), (0, 2)]
        assert maze_solver.solve() == solution


## Task 1.2
The visualization of the paths found for the mazes `board-1-{1,2,3,4}` are now shown:

In [18]:
# %load main.py
from mazesolver import MazeSolver

if __name__ == '__main__':
    board_paths = 'mazesolver/tests/boards/board-1-'

    for board_number in [1, 2, 3, 4]:
        with open(board_paths + str(board_number) + '.txt') as board_file:
            maze_solver = MazeSolver(board_file.read())
            maze_solver.visualize()

----------------------
|....................|
|....................|
|.........######.....|
|........OOOO..#..O..|
|........O######..O..|
|........OOOOOOOOOO..|
|....................|
----------------------
----------------------
|....OOO#............|
|...OO#O#............|
|..OO#OO#............|
|OOO#.O#....OOOOOOOOO|
|....#OO#..OO#.......|
|.....#OO#OO#........|
|......#OOO#.........|
----------------------
----------------------
|.........OOO........|
|.........O#OO.......|
|.......##OO#OOOO....|
|......#OO#O#...O....|
|......#O#OO#...O....|
|......#OOO#....OO...|
|.......###......OOOO|
----------------------
----------------------
|OO#.......#......#..|
|#O#.#####.#.####.#..|
|OO#OOOOO#.#....#....|
|O##O###O######.#####|
|OO#OO#OO#....#...#..|
|#O####O##.##.#.#.##.|
|.OOOOOO....#...#....|
----------------------
